# Steps for Causal Impact

1. Define the Pre and Post Period.

2. Retrieve the data we need.

3. Check whether the variables are correlated in the pre-period.

4. Remove non-correlted data.

5. Use Causal Impact.

# Loading the necessary libraries


- **yfinance** library helps us retrieve financial data from yahoo finance.

- **tfcausalimpact** is Google's Causal Impact Algorithm Implemented on Top of TensorFlow Probability.

- **TensorFlow Probability** is a library for probabilistic reasoning and statistical analysis in TensorFlow.

- **Causal Impact** is an R package for causal inference using Bayesian structural time-series models.

In [ ]:
# Loading the necessary libraries
!pip install yfinance
!pip install tfcausalimpact

In [2]:
# Import the libraries
import yfinance as yf
from causalimpact import CausalImpact
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# Define Dates
training_start = "2020-09-01" # Had a ratio of 10:1, training is of 40 days and the treatment is of 4 days
training_end = "2020-10-19"
treatment_start = "2020-10-20"
treatment_end = "2020-10-23"
end_stock = "2020-10-24" # Always add an additional day from treatment_end

# Loading Financial Data

In [4]:
# Bitcoin Data
y = ["BTC-USD"]
y = yf.download(tickers = y,
                start = training_start,
                end = end_stock,
                interval = "1d")
y

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-09-01,11679.316406,12067.081055,11601.128906,11970.478516,11970.478516,27311555343
2020-09-02,11964.823242,11964.823242,11290.793945,11414.034180,11414.034180,28037405299
2020-09-03,11407.191406,11443.022461,10182.464844,10245.296875,10245.296875,31927261555
2020-09-04,10230.365234,10663.919922,10207.940430,10511.813477,10511.813477,29965130374
2020-09-05,10512.530273,10581.571289,9946.675781,10169.567383,10169.567383,44916565292
2020-09-06,10167.216797,10353.927734,10056.885742,10280.351562,10280.351562,37071460174
2020-09-07,10280.998047,10399.153320,9916.493164,10369.563477,10369.563477,33703098409
2020-09-08,10369.306641,10414.775391,9945.110352,10131.516602,10131.516602,33430927462
2020-09-09,10134.151367,10350.542969,10017.250977,10242.347656,10242.347656,24128292755


- Out of all these columns we need **Adj Close** so we will get that by doing.

In [5]:
y = y['Adj Close']

In [6]:
y

Date
2020-09-01    11970.478516
2020-09-02    11414.034180
2020-09-03    10245.296875
2020-09-04    10511.813477
2020-09-05    10169.567383
2020-09-06    10280.351562
2020-09-07    10369.563477
2020-09-08    10131.516602
2020-09-09    10242.347656
2020-09-10    10363.138672
2020-09-11    10400.915039
2020-09-12    10442.170898
2020-09-13    10323.755859
2020-09-14    10680.837891
2020-09-15    10796.951172
2020-09-16    10974.905273
2020-09-17    10948.990234
2020-09-18    10944.585938
2020-09-19    11094.346680
2020-09-20    10938.271484
2020-09-21    10462.259766
2020-09-22    10538.459961
2020-09-23    10246.186523
2020-09-24    10760.066406
2020-09-25    10692.716797
2020-09-26    10750.723633
2020-09-27    10775.269531
2020-09-28    10709.652344
2020-09-29    10844.640625
2020-09-30    10784.491211
2020-10-01    10619.452148
2020-10-02    10575.974609
2020-10-03    10549.329102
2020-10-04    10669.583008
2020-10-05    10793.339844
2020-10-06    10604.406250
2020-10-07    10668.968

In [8]:
type(y)

pandas.core.series.Series

- Rename the data array like this

In [9]:
y = y.rename("y")
y.tail()

Date
2020-10-19    11742.037109
2020-10-20    11916.334961
2020-10-21    12823.689453
2020-10-22    12965.891602
2020-10-23    12931.539062
Name: y, dtype: float64

- Training Group is nothing but **Control Group**.

- Here the treatment and training group are assumed to have same KPI.

- More the number of training groups precise the analysis would be.

- Try keeping the post-period (treatment duration) to bare minimum.

In [15]:
# Loading the Stock Data
stocks = ["CARL-B.CO", "ZAL.DE", "SQ", "CRSP", "TRMB", "JD", "DE", "KTOS", "GOOG"]
x = yf.download(tickers = stocks,
                start = training_start,
                end = end_stock,
                interval = '1d')

x.head()

[*********************100%%**********************]  9 of 9 completed


Adj Close                         ...      Volume                   
             CARL-B.CO       CRSP          DE  ...          SQ       TRMB  ZAL.DE
Date                                           ...                               
2020-09-01  794.451111  93.419998  209.291962  ...  12306400.0   542300.0  790198
2020-09-02  802.765198  93.930000  208.734344  ...  11214800.0   728800.0  496554
2020-09-03  818.099915  85.690002  202.677383  ...  16421200.0  1220300.0  736259
2020-09-04  804.612732  82.019997  203.186951  ...  17995200.0   914700.0  662250
2020-09-07  812.926697        NaN         NaN  ...         NaN        NaN  362062

[5 rows x 54 columns]

In [16]:
# Getting just the required data
x = x.iloc[:,:len(stocks)]
x.head(1)

Adj Close                         ...                          
             CARL-B.CO       CRSP          DE  ...          SQ   TRMB ZAL.DE
Date                                           ...                          
2020-09-01  794.451111  93.419998  209.291962  ...  166.660004  53.41   77.0

[1 rows x 9 columns]

In [18]:
x.columns = x.columns.droplevel()
x.head()

,CARL-B.CO,CRSP,DE,GOOG,JD,KTOS,SQ,TRMB,ZAL.DE
Date,,,,,,,,,
2020-09-01,794.451111,93.419998,209.291962,83.035500,79.267685,19.700001,166.660004,53.410000,77.000000
2020-09-02,802.765198,93.930000,208.734344,86.414001,79.959564,19.910000,162.880005,54.310001,77.000000
2020-09-03,818.099915,85.690002,202.677383,82.092003,75.952454,19.520000,152.860001,50.900002,75.080002
2020-09-04,804.612732,82.019997,203.186951,79.552002,76.874954,19.260000,146.389999,49.959999,71.739998
2020-09-07,812.926697,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.000000


In [19]:
# Combine all the data
df = pd.concat([y, x], axis=1).dropna()
df.head()

,y,CARL-B.CO,CRSP,DE,GOOG,JD,KTOS,SQ,TRMB,ZAL.DE
Date,,,,,,,,,,
2020-09-01,11970.478516,794.451111,93.419998,209.291962,83.035500,79.267685,19.700001,166.660004,53.410000,77.000000
2020-09-02,11414.034180,802.765198,93.930000,208.734344,86.414001,79.959564,19.910000,162.880005,54.310001,77.000000
2020-09-03,10245.296875,818.099915,85.690002,202.677383,82.092003,75.952454,19.520000,152.860001,50.900002,75.080002
2020-09-04,10511.813477,804.612732,82.019997,203.186951,79.552002,76.874954,19.260000,146.389999,49.959999,71.739998
2020-09-08,10131.516602,799.439575,81.459999,202.129395,76.619499,73.242615,19.230000,139.110001,49.139999,73.440002
